In [26]:
from mewpy.simulation import solvers
from mewpy.simulation import set_default_solver
from cobra.io.sbml import read_sbml_model
set_default_solver('glpk')
import pandas as pd
from mewpy.simulation import get_simulator

Set parameter Username


In [4]:
%store -r ocm_t #dataframe with growths in first column and gene expression in subsequent columns, OCM IDs in rows
%store -r ocmi_2 #media condition dictionary defined elsewhere
%store -r ocmi_essential_reactions #'essential_reactions' function within mewpy.simulator package
%store -r ocmi_essential_genes # 'essential_genes' function within mewpy.simulator package

In [5]:
#building function to build constraint-based model per OCM...
def constrain_model_2(row_number):
    #import model and print media-only solution
    model = read_sbml_model('Human-GEM-annotated.xml') #can be changed for any model, but need to match dataset gene ids and model gene ids
    media = ocmi_2 # a dictionary, which depends on the media in which your cells have been grown
    media_only_simulation = get_simulator(model, envcond = media)
    media_only_result = media_only_simulation.simulate()
    print(media_only_result)

    #import rnaseq data, and select row for analysis
    gene_exp = ocm_t_2.iloc[row_number,:] #dataframe where I have my rnaseq data, samples as rows, ensembl gene ids as columns
    g_list = []
    for k in gene_exp.keys():
        g_list.append(k)
    vs = []
    for v in gene_exp:
        vs.append(v)
    gene_exp_dict = {}
    for n in range(len(g_list)):
        gene_exp_dict[g_list[n]] = vs[n]

    #initial separation of model according to reaction rules
    ANDs = []
    ORs = []
    ANDORs = []
    one_gene = []
    no_gene = []
    for r in model.reactions:
        if 'and' in r.gene_reaction_rule and 'or' not in r.gene_reaction_rule:
            ANDs.append(r.id)
        if 'and' in r.gene_reaction_rule and 'or' in r.gene_reaction_rule:
            ANDORs.append(r.id)
        if 'or' in r.gene_reaction_rule and 'and' not in r.gene_reaction_rule:
            ORs.append(r.id)
        if len(r.gene_reaction_rule) == 0:
            no_gene.append(r.id)
        if len(r.gene_reaction_rule) != 0:
            if 'or' in r.gene_reaction_rule:
                continue
            elif 'and' in r.gene_reaction_rule:
                continue
            else:
                one_gene.append(r.id)
    print('AND rules: ', len(ANDs))
    print('ANDOR rules: ', len(ANDORs))
    print('OR rules: ', len(ORs))
    print('ONE GENE rules: ', len(one_gene))
    print('NO GENE rules: ', len(no_gene))
    print('Proportion of model not annotated: ', len(no_gene)/len(model.reactions)*100, '%')
    print('Proportion of model which IS annotated: ', 100-(len(no_gene)/len(model.reactions)*100), '%')
    print('Total Reactions = ', len(model.reactions))
    if len(model.reactions) == (len(ORs)+len(ANDs)+len(ANDORs)+len(one_gene)+len(no_gene)):
        print('model has been completely, successfuly separated')
    else:
        print('some reactions have not been grouped ready for analysis...')
        
    #separate model further, according to reversibility of rule
    one_gene_forward = []
    one_gene_reversible = []
    for r in one_gene:
        if model.reactions.get_by_id(r).reversibility == True:
            one_gene_reversible.append(r)
        else:
            one_gene_forward.append(r)
    print('number of reversible, one gene reactions =', len(one_gene_reversible))
    print('number of forward, one gene reactions =', len(one_gene_forward))
    or_forward = []
    or_reversible = []
    for r in ORs:
        if model.reactions.get_by_id(r).reversibility == True:
            or_reversible.append(r)
        else:
            or_forward.append(r)
    print('number of reversible, OR reactions =', len(or_reversible))
    print('number of forward, OR reactions =', len(or_forward))
    and_forward = []
    and_reversible = []
    for r in ANDs:
        if model.reactions.get_by_id(r).reversibility == True:
            and_reversible.append(r)
        else:
            and_forward.append(r)
    print('number of reversible, AND reactions =', len(and_reversible))
    print('number of forward, AND reactions =', len(and_forward))
          
    #constrain model according to reaction rules
    one_gene_constrained_reactions = {}
    for r in one_gene_forward:
        if r not in media.keys():
            if r not in ocmi_essential_reactions:
                if model.reactions.get_by_id(r).gene_reaction_rule not in ocmi_essential_genes:
                    if model.reactions.get_by_id(r).gene_reaction_rule in gene_exp_dict.keys(): 
                        if gene_exp_dict[model.reactions.get_by_id(r).gene_reaction_rule] != (0,0):
                            model.reactions.get_by_id(r).bounds = (0,(gene_exp_dict[model.reactions.get_by_id(r).gene_reaction_rule]))
                            solution = model.optimize()
                            if solution.fluxes['biomass_human'] <= ocm_t_2.iloc[row_number,0]:
                                print(r, ':', 'constrained bounds', model.reactions.get_by_id(r).bounds, solution.fluxes['biomass_human'])
                                model.reactions.get_by_id(r).bounds = (0,1000)
                                solution = model.optimize()
                                print(r, ':', 're-opened bounds', model.reactions.get_by_id(r).bounds, solution.fluxes['biomass_human'])
                                one_gene_constrained_reactions[r] = model.reactions.get_by_id(r).bounds
                            if solution.fluxes['biomass_human'] != 0:
                                if solution.fluxes['biomass_human'] > ocm_t_2.iloc[row_number,0]:
                                    print(r, ':', model.reactions.get_by_id(r).bounds, solution.fluxes['biomass_human'])
                                    one_gene_constrained_reactions[r] = model.reactions.get_by_id(r).bounds
    for r in one_gene_reversible:
        if r not in media.keys():
            if r not in ocmi_essential_reactions: 
                if model.reactions.get_by_id(r).gene_reaction_rule not in ocmi_essential_genes:
                    if model.reactions.get_by_id(r).gene_reaction_rule in gene_exp_dict.keys(): 
                        if gene_exp_dict[model.reactions.get_by_id(r).gene_reaction_rule] != (0,0): 
                            model.reactions.get_by_id(r).bounds = (-1*(gene_exp_dict[model.reactions.get_by_id(r).gene_reaction_rule]),(gene_exp_dict[model.reactions.get_by_id(r).gene_reaction_rule]))
                            solution = model.optimize()
                            if solution.fluxes['biomass_human'] <= ocm_t_2.iloc[row_number,0]:
                                print(r, ':', 'constrained bounds', model.reactions.get_by_id(r).bounds, solution.fluxes['biomass_human'])
                                model.reactions.get_by_id(r).bounds = (-1000,1000)
                                solution = model.optimize()
                                print(r, ':', 're-opened bounds', model.reactions.get_by_id(r).bounds, solution.fluxes['biomass_human'])
                                one_gene_constrained_reactions[r] = model.reactions.get_by_id(r).bounds
                            if solution.fluxes['biomass_human'] != 0:
                                if solution.fluxes['biomass_human'] > ocm_t_2.iloc[row_number,0]:
                                    print(r, ':', model.reactions.get_by_id(r).bounds, solution.fluxes['biomass_human'])
                                    one_gene_constrained_reactions[r] = model.reactions.get_by_id(r).bounds
    for r in or_forward:
        if r not in media.keys():
            if r not in ocmi_essential_reactions:
                rule_list = []
                rule = model.reactions.get_by_id(r).gene_reaction_rule
                rule_list.append(rule.split())
                rule_genes = []
                for n in rule_list:
                    for num in n:
                        if 'ENS' in num:
                            rule_genes.append(num)
                #calculate sum of expressions
                genes_in_dataset = []
                for gene in rule_genes:
                    if gene in gene_exp_dict.keys():
                        genes_in_dataset.append(gene)
                exp_list = []        
                for g in genes_in_dataset:
                    exp_list.append(gene_exp_dict[g])
                sum_of_expressions = sum(exp_list)
                #set bounds according to sum of expressions
                model.reactions.get_by_id(r).bounds = (0,sum_of_expressions)
                solution = model.optimize()
                if solution.fluxes['biomass_human'] <= ocm_t_2.iloc[row_number,0]:
                    print(r, ':', 'constrained bounds', model.reactions.get_by_id(r).bounds, solution.fluxes['biomass_human'])
                    model.reactions.get_by_id(r).bounds = (0,1000)
                    solution = model.optimize()
                    print(r, ':', 're-opened bounds', model.reactions.get_by_id(r).bounds, solution.fluxes['biomass_human'])
                    one_gene_constrained_reactions[r] = model.reactions.get_by_id(r).bounds
                if solution.fluxes['biomass_human'] != 0:
                    if solution.fluxes['biomass_human'] > ocm_t_2.iloc[row_number,0]:
                        print(r, ':', model.reactions.get_by_id(r).bounds, solution.fluxes['biomass_human'])
                        one_gene_constrained_reactions[r] = model.reactions.get_by_id(r).bounds
    for r in or_reversible:
        if r not in media.keys():
            if r not in ocmi_essential_reactions:
                rule_list = []
                rule = model.reactions.get_by_id(r).gene_reaction_rule
                rule_list.append(rule.split())
                rule_genes = []
                for n in rule_list:
                    for num in n:
                        if 'ENS' in num:
                            rule_genes.append(num)
                #calculate sum of expressions
                genes_in_dataset = []
                for gene in rule_genes:
                    if gene in gene_exp_dict.keys():
                        genes_in_dataset.append(gene)
                exp_list = []        
                for g in genes_in_dataset:
                    exp_list.append(gene_exp_dict[g])
                sum_of_expressions = sum(exp_list)
                #set bounds according to sum of expressions
                model.reactions.get_by_id(r).bounds = (-1*sum_of_expressions,sum_of_expressions)
                solution = model.optimize()
                if solution.fluxes['biomass_human'] <= ocm_t_2.iloc[row_number,0]:
                    print(r, ':', 'constrained bounds', model.reactions.get_by_id(r).bounds, solution.fluxes['biomass_human'])
                    model.reactions.get_by_id(r).bounds = (-1000,1000)
                    solution = model.optimize()
                    print(r, ':', 're-opened bounds', model.reactions.get_by_id(r).bounds, solution.fluxes['biomass_human'])
                    one_gene_constrained_reactions[r] = model.reactions.get_by_id(r).bounds
                if solution.fluxes['biomass_human'] != 0:
                    if solution.fluxes['biomass_human'] > ocm_t_2.iloc[row_number,0]:
                        print(r, ':', model.reactions.get_by_id(r).bounds, solution.fluxes['biomass_human'])
                        one_gene_constrained_reactions[r] = model.reactions.get_by_id(r).bounds
    for r in and_forward:
        if r not in media.keys():
            if r not in ocmi_essential_reactions:
                rule_list = []
                rule = model.reactions.get_by_id(r).gene_reaction_rule
                rule_list.append(rule.split())
                rule_genes = []
                for n in rule_list:
                    for num in n:
                        if 'ENS' in num:
                            rule_genes.append(num)
                #calculate sum of expressions
                genes_in_dataset = []
                for gene in rule_genes:
                    if gene in gene_exp_dict.keys():
                        genes_in_dataset.append(gene)
                exp_list = []        
                for g in genes_in_dataset:
                    exp_list.append(gene_exp_dict[g])
                if len(exp_list) > 1:
                    exp_list.sort()
                    min_expression = exp_list[0]
                if len(exp_list) == 1:
                    min_expression = exp_list[0]
                #set bounds according to sum of expressions
                model.reactions.get_by_id(r).bounds = (0,min_expression)
                solution = model.optimize()
                if solution.fluxes['biomass_human'] <= ocm_t_2.iloc[row_number,0]:
                    print(r, ':', 'constrained bounds', model.reactions.get_by_id(r).bounds, solution.fluxes['biomass_human'])
                    model.reactions.get_by_id(r).bounds = (0,1000)
                    solution = model.optimize()
                    print(r, ':', 're-opened bounds', model.reactions.get_by_id(r).bounds, solution.fluxes['biomass_human'])
                    one_gene_constrained_reactions[r] = model.reactions.get_by_id(r).bounds
                if solution.fluxes['biomass_human'] != 0:
                    if solution.fluxes['biomass_human'] > ocm_t_2.iloc[row_number,0]:
                        print(r, ':', model.reactions.get_by_id(r).bounds, solution.fluxes['biomass_human'])
                        one_gene_constrained_reactions[r] = model.reactions.get_by_id(r).bounds
    for r in and_reversible:
        if r not in media.keys():
            if r not in ocmi_essential_reactions:
                rule_list = []
                rule = model.reactions.get_by_id(r).gene_reaction_rule
                rule_list.append(rule.split())
                rule_genes = []
                for n in rule_list:
                    for num in n:
                        if 'ENS' in num:
                            rule_genes.append(num)
                #calculate sum of expressions
                genes_in_dataset = []
                for gene in rule_genes:
                    if gene in gene_exp_dict.keys():
                        genes_in_dataset.append(gene)
                exp_list = []        
                for g in genes_in_dataset:
                    exp_list.append(gene_exp_dict[g])
                if len(exp_list) > 1:
                    exp_list.sort()
                    min_expression = exp_list[0]
                if len(exp_list) == 1:
                    min_expression = exp_list[0]
                #set bounds according to sum of expressions
                model.reactions.get_by_id(r).bounds = ((-1*min_expression),min_expression)
                solution = model.optimize()
                if solution.fluxes['biomass_human'] <= ocm_t_2.iloc[row_number,0]:
                    print(r, ':', 'constrained bounds', model.reactions.get_by_id(r).bounds, solution.fluxes['biomass_human'])
                    model.reactions.get_by_id(r).bounds = (-1000,1000)
                    solution = model.optimize()
                    print(r, ':', 're-opened bounds', model.reactions.get_by_id(r).bounds, solution.fluxes['biomass_human'])
                    one_gene_constrained_reactions[r] = model.reactions.get_by_id(r).bounds
                if solution.fluxes['biomass_human'] != 0:
                    if solution.fluxes['biomass_human'] > ocm_t_2.iloc[row_number,0]:
                        print(r, ':', model.reactions.get_by_id(r).bounds, solution.fluxes['biomass_human'])
                        one_gene_constrained_reactions[r] = model.reactions.get_by_id(r).bounds
    
    #print biomass solution for final constrained model
    all_and_simulation = get_simulator(model, envcond = media, constraints = one_gene_constrained_reactions) #one_gene_constrained_reactions
    all_and_result = all_and_simulation.simulate()
    print(all_and_result)
    print('growth after sample constraints (hours):', 1/(all_and_result.find(['biomass_human'])))
    
    #save final constrained reaction fluxes as dataframe
    ids = []
    fluxes = []
    for r in model.reactions:
        if 'EX' not in r.id:
            ids.append(r.id)
            fluxes.append(all_and_result.find([r.id]).iloc[0,0])
    column_names = ['reaction', 'flux']
    fluxes_df = pd.DataFrame(columns = column_names)
    fluxes_df['reaction'] = ids
    fluxes_df['flux'] = fluxes
    return(fluxes_df)

In [24]:
#find mean of ocm gene expression data (to compare to ccle and adjust if necessary, similar to when proteomics was transformed)
row_zero = []
for n in ocm_t.iloc[0,1:]:
    row_zero.append(n)
    
sum_row_zero = sum(row_zero)
mean_row_zero_ocm = sum_row_zero/len(row_zero)
print('mean of first row ocm data:',mean_row_zero_ocm)

#compare to ccle 2020 tomics
%store -r t
row_zero = []
for n in t.iloc[0,:]:
    row_zero.append(n)
    
sum_row_zero = sum(row_zero)
mean_row_zero_ccle = sum_row_zero/len(row_zero)
print('mean of first row ccle data:',mean_row_zero_ccle)

print('factor difference in means for first OCM is:',mean_row_zero_ocm/mean_row_zero_ccle)

mean of first row ocm data: 5.72791381812603
mean of first row ccle data: 1.0407701621111483
factor difference in means for first OCM is: 5.50353385084297


In [27]:
#save to csv and divide all values by 5 in Excel to adjust spread
ocm_t.to_csv('ocm_t_2.csv')
#upload adjusted ocm gene expression data
ocm_t_2 = pd.read_csv(r'/Users/katemeeson/Library/Mobile Documents/com~apple~CloudDocs/MRC-DTP_PhD_UoM/OCM_work/datasets/bc_rnaseq_with_growths_adjusted.csv')
ocm_t_2 = ocm_t_2.set_index('Unnamed: 0')
ocm_t_2 = ocm_t_2.transpose()

In [83]:
#an example of using function to constrain model to OCM-specific
#the row index (here is zero) is changed each time, and result is saved as variable
OCM46_3_T_P4 = constrain_model_2(0)
OCM46_3_T_P4

objective: 0.0858850564569515
Status: OPTIMAL
Constraints: OrderedDict([('HMR_9063', (-0.438, 1000)), ('HMR_9169', (-8e-06, 1000)), ('HMR_9314', (-5e-07, 1000)), ('HMR_9358', (-0.00933, 1000)), ('HMR_9167', (-0.00026, 1000)), ('HMR_9285', (-0.00205, 1000)), ('HMR_9158', (-0.000512, 1000)), ('HMR_9151', (-0.0001, 1000)), ('HMR_9146', (-0.00175, 1000)), ('HMR_9441', (-1.000025, 1000)), ('HMR_9442', (-2.5e-05, 1000)), ('HMR_9361', (-0.023, 1000)), ('HMR_9437', (-0.003075, 1000)), ('HMR_9406', (-0.005125, 1000)), ('EX_xan[e]', (-0.003525, 1000)), ('HMR_9067', (-0.50751, 1000)), ('HMR_9061', (-0.259, 1000)), ('HMR_9066', (-0.911, 1000)), ('HMR_9070', (-0.3133, 1000)), ('HMR_9065', (-0.0361, 1000)), ('HMR_9363', (-0.26, 1000)), ('HMR_9071', (-0.7647, 1000)), ('HMR_9038', (-0.23976, 1000)), ('EX_4hpro[e]', (-0.1, 1000)), ('HMR_9039', (-0.40394, 1000)), ('HMR_9040', (-0.6131, 1000)), ('HMR_9041', (-0.7365, 1000)), ('HMR_9042', (-0.15448, 1000)), ('HMR_9043', (-0.25496, 1000)), ('HMR_9068', (-0

,reaction,flux
0,HMR_3905,0.0
1,HMR_3907,0.0
2,HMR_4097,0.0
3,HMR_4099,0.0
4,HMR_4108,0.0
...,...,...
11870,HMR_10128,0.0
11871,HMR_10129,0.0
11872,HMR_10130,0.0
11873,HMR_10131,0.0


In [85]:
#save result as csv
OCM46_3_T_P4.to_csv('OCM46_3_T_P4_adjusted.csv')
#store result as variable
%store OCM46_3_T_P4